In [1]:
import sys
import math
from functools import reduce
from textblob import TextBlob
from textblob import Word
from collections import defaultdict

In [2]:
postingList = defaultdict(dict)
doc_num = 0
K=100
d_num = defaultdict(int)#文档数组
t_num = defaultdict(int)#词数组
score_tid = defaultdict(dict)
Term = ["tweetid", "usename", "text"]

In [3]:
def tokenize_(document):
    document = document.lower()
    a = document.index("username")
    b=document.index("clusterno")
    c=document.rindex("tweetid") - 1
    d=document.rindex("errorcode")
    e=document.index("text")
    f=document.index("timestr") - 3
    #获取tweetid、username、text这三部分信息
    document=document[c:d] + document[a:b] + document[e:f]  
    #将所有名词变为单数形式
    terms=TextBlob(document).words.singularize()
    res=[]
    for word in terms:
        t = Word(word)
        t = t.lemmatize("v")#将动词恢复原形
        if t not in Term:
            res.append(t)
    return res

In [4]:
def token(document):
    document = document.lower()
    terms = TextBlob(document).words.singularize()
    res=[]
    for word in terms:
        t = Word(word)
        t = t.lemmatize("v")
        res.append(t)
    return res

In [5]:
def get_posting():
    global postingList, doc_num
    f = open(r"tweets.txt")
    lines = f.readlines()
    doc_num=0
    for line in lines:
        doc_num = doc_num + 1
        line = tokenize_(line)
        tweetid = line[0]
        line.pop(0)
        d_num={}
        for term in line:
            if term in d_num.keys():
                d_num[term]+=1
            else:
                d_num[term]=1
        for term in d_num.keys():
            d_num[term] = 1 + math.log10(d_num[term])
        #normalization
        nor=0
        for term in d_num.keys():
            nor = nor + d_num[term]
        nor = 1.0 / math.sqrt(nor)
        for term in d_num.keys():
            d_num[term] = d_num[term] * nor
        #sort
        uniTerm = set(line)
        for term in uniTerm:
            if term in postingList.keys():
                postingList[term][tweetid] = d_num[term]
            else:
                postingList[term][tweetid] = d_num[term]

In [6]:
def Union(sets):
    return reduce(set.union, [s for s in sets])

In [7]:
def similarity_(query):
    global socre_tid
    t_num={}#存放query中每个词的频数
    #计算query中每个词出现的次数
    for term in query:
        if term in t_num:
            t_num[term]+=1
        else:
            t_num[term]=1
    for term in t_num.keys():
        if term in postingList:
            d_f = len(postingList[term])
        else:
            d_f = doc_num
        t_num[term] = (math.log10(t_num[term])+1) * math.log10(doc_num/d_f)
    for term in query:
        if term in postingList:
            for tid in postingList[term]:
                if tid in score_tid.keys():
                    score_tid[tid] += postingList[term][tid] * t_num[term]
                else:
                    score_tid[tid] = postingList[term][tid] * t_num[term]
    similarity = sorted(score_tid.items(), key=lambda x:x[1], reverse=True)
    return similarity

In [8]:
def do_search():
    query = token(input("please input the query : "))
    res=[]
    if query==[]:
        sys.exit()
    uniquery = set(query)
    relevant_tid = Union([set(postingList[term].keys()) for term in uniquery])
    print("------------------Start------------------")
    print("There are "+str(len(relevant_tid))+" relevant tweets in total!")
    if not relevant_tid:
        print("There is no relevant tweet!")
    else:
        print("( TweetId : Score )")
        print("The top "+str(K)+" tweets are: ")
        scores = similarity_(query)
        i=1
        for (id,score) in scores:
            if i<=K:
                res.append(id)
                print(id+" : "+str(score))
                i = i+1
            else:
                break
        print("------------------Finished!------------------")

In [9]:
get_posting()

In [ ]:
do_search()